## Background

Over the last few years, one of the places in SunPy I have been focusing my development efforts has been improving SunPy's coordinate handling. This has really taken two main tracks, the first is the better handling of FITS-WCS in SunPy Map, the second, and the topic of this post, is the representation of physical coordinates in SunPy.
The full story of this post really starts before the 2013 Astropy coordination meeting, which I attended remotely. It was at this meeting I discussed with Erik Tollerud about the future redesign of Astropy's coordinates module and how we could leverage this for SunPy. Skip forward a fair amount and after all the planning and implementing of Astropy coordinates (which I was involved with), we set about implementing the solar physics coordinate systems in SunPy but as a 'plugin' to Astropy's module.

To do this I had the invaluble help of a Google Summer of Code student [Pritish Chakraborty](http://pritishc.com/) who did the vast majority of the implementation of what is now the `sunpy.coordinates` module. He and I embarked on a mission to learn not just the Astropy coordinates package which we were using, but also all the maths of these systems and the transformations between them, which will be the topic of this post.

Anyway, enough of the back story and onto the coordinates!

# Solar Coordinate Systems 

In [ ]:
import copy

%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import InsetPosition, BboxConnector
import matplotlib.animation as anim
import sunpy.coordinates
from astropy.visualization import wcsaxes

from astropy.wcs import WCS
import numpy as np

## Terminology and Basics

Before we progress any further we must clarify what a coordinate system is and what we are going to call things. A 'coordinate system' is a way of describing points in two dimensional or three dimensional space which is well defined. This 'system' comprises of two main components:

* A representation of a point in vector space, i.e. cartesian, spherical or cylindrical.
* A reference system, which orientates the axes of the system with respect to some well defined points.

We adopt this convention from [APE 5](https://github.com/astropy/astropy-APEs/blob/master/APE5.rst#backgroundterminology).

For the rest of this post we shall mainly be discussing reference systems and the different ones commonly in use in solar physics.

## Projected Coordinate Systems

One common type of coordinate system in astronomy is the projected coordinate system. When observing the sky, the positions of objects on the sky can be described as points on a two dimensional spherical coordinate system, centred upon the observer. This also applies to imaging the sky, as most astronomers do regularly.

Like any other coordinate system a projected system has to have an origin, which is the observer, and it has to be aligned to some reference axes. In the case of the solar physics projected coordinate systems these reference axes are the line from the solar centre to the solar north pole, and the line from the solar centre to the observer.

There are two solar-specifc projected coordinate systems, helioprojective-cartesian and helioprojective-radial. The first, and most important thing to point out is that, despite the misleading names given to them, they are both fully sperical coordinate systems with positions given as longitude and latitude on the celestial sphere. They are so named, because of their similarities to the true 3D heliocentric coordinate systems we shall discuss later.


### Helioprojective-Cartesian Frame

The most commonly used coordinate frame in solar physics is heliocentric-cartesian, this frame has the zero line of longitude aligned with the line from the solar centre to the solar north pole. It also has the line of zero latitude aligned with the solar equator, meaning that the point of zero longitude and latitude is given to be the centre of the solar disk as seen by the observer.

Due to the small angular size of the Sun as seen from Earth, just under $2000''$ or just over $0.55^\circ$, it has been historically common to approximate this coordinate system as a cartesian system with the y axis being latitude and the x axis being longitude. This is possible due to the small angle approximation, which states that $\sin \theta \approx \theta$ and $\cos \theta \approx 1 - \frac{\theta^2}{2}$ the error in these approximations is well below 1% for any point on the solar disk.

This historical convention has lead to the common labelling of helioprojective longitude and latitude as solar-x and solar-y, and the notion that HPC (helioprojective-cartesian) is actually not a spherical system, which is false.

In [ ]:
blank = np.zeros((2048, 2048)) * np.NaN

hpc_wcs = WCS(naxis=2)
hpc_wcs.wcs.ctype = ['HPLN-TAN', 'HPLT-TAN']
hpc_wcs.wcs.crval = [0.]*2
hpc_wcs.wcs.crpix = np.array(blank.shape)/2
hpc_wcs.wcs.cdelt = [400.]*2
hpc_wcs.wcs.cunit = ['arcsec']*2

hpc_wcs = WCS(naxis=2)
hpc_wcs.wcs.ctype = ['HPLN-ZEA', 'HPLT-ZEA']
hpc_wcs.wcs.crval = [0., 0.]
hpc_wcs.wcs.crpix = np.array(blank.shape)/2
hpc_wcs.wcs.cdelt = [250.,250.]
hpc_wcs.wcs.cunit = ['arcsec']*2

inset_wcs = WCS(naxis=2)
inset_wcs.wcs.ctype = ['HPLN-TAN', 'HPLT-TAN']
inset_wcs.wcs.crval = [0.]*2
inset_wcs.wcs.crpix = np.array(blank.shape)/2
inset_wcs.wcs.cdelt = [4.]*2
inset_wcs.wcs.cunit = ['arcsec']*2

fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, projection=hpc_wcs)
ax.imshow(blank, origin='lower')

x = ax.coords[0]
y = ax.coords[1]

x.set_axislabel("Longitude [deg]")
y.set_axislabel("Latitude [deg]")

ax.coords.grid(color='black', ls='-')

inset_axes = plt.axes([0, 0, 1, 1], projection=inset_wcs)
ip = InsetPosition(ax, [0.73, 0.73, 0.25, 0.25])
inset_axes.set_axes_locator(ip)


inset_axes.imshow(blank, origin='lower')
x = inset_axes.coords[0]
y = inset_axes.coords[1]

x.set_ticks(number=4)
y.set_ticks(number=4)

x.set_major_formatter('s.s')
y.set_major_formatter('s.s')

x.set_axislabel("Longitude [arcsec]")
y.set_axislabel("Latitude [arcsec]")

inset_axes.coords.grid(color='black', ls='-')

circ = plt.Circle(inset_wcs.wcs.crpix, radius=(1200/3600)/inset_wcs.wcs.cdelt[0], color='yellow', ec='k')
inset_axes.add_artist(circ)

width, height = np.array(blank.shape) * inset_wcs.wcs.cdelt[0]
centre = (0-width/2, 0-height/2)
rect = plt.Rectangle(centre, width, height, transform=ax.get_transform('world'), fc='none')
ax.add_artist(rect)

p1 = BboxConnector(inset_axes.bbox, rect, loc1=2, ls='--')
inset_axes.add_patch(p1)
p1.set_clip_on(False)
p2 = BboxConnector(inset_axes.bbox, rect, loc1=4, ls='--')
inset_axes.add_patch(p2)
p2.set_clip_on(False)

t = ax.set_title("Demonstrating the cartesian approximation")

### Helioprojective-Radial Frame

The helioprojective-radial (HPR) frame is very similar to the helioprojective-cartesian (HPC) frame apart from the point on the celestial sphere that is aligned to the centre of the solar disk. While the HPC system sets the centre of the disk to be the point (0,0) of longitude and latitude, the HPR system defines it to be (0, -90), i.e. at the south pole of the celestial sphere.

As with HPC there is a confusing convention normally adopted when using HPR. By definition the south pole of the celestial sphere actually has the coordinates (0, 0) in HPR and latitude goes from $0^\circ$ to $180^\circ$, rather than from $90^\circ$ to $-90^\circ$ as is usual in spherical coordinates.

## Heliocentric Systems

The other commonly used systems in solar physics have the centre of the Sun as the origin, thus heliocentric, as opposed to observer centric. Unlike the projective systems, which are normally used as two dimensional (the radius being technically at infinity), the heliocentric systems are fully three dimensional and are always used as such.

### Heliocentric (Cartesian)

The so called 'Heliocentric' system is observer aligned, the two axes are the line between the centre of the Sun and the solar north pole and the line from the centre of the Sun to the observer. You may recall this is the same as the Helioprojective-Cartesian system, only differing in the choice of origin. This is where the somewhat confusing name of the helioprojective-cartesian frame originates from, because if you take the small angle approximation, cartesian coordinates in the helioprojective-cartesian system can be approximated to coordinates in the heliocentric system.

The heliocentric system, due to its similarities to the helioprojective-cartesian system is very useful for conversion from helioprojective-cartesian to points on the solar surface. Obviously, when converting from a projective (and two dimensional) point into a heliocentric (and three dimensional) system, an assumption has to be made to recover the third coordinate. This assumption is that the coordinate points lay on the surface of a sphere of fixed radius, normally taken to be the photospheric radius of the Sun. Using this assumption, the poisition in three dimensional space can be calculated. This step is performed automatically by the `sunpy.coordinates` module if the helioprojective frame is specified without a third coordinate.

### Heliographic (Stonyhurst)

The third major type of coordinate system used is the heliographic type, of which there is two variations. Heliographic systems are spherical heliocentric systems and are commonly used to specify points on the solar surface in a way which is not observer dependent. The two axes of the heliographic-stonyhurst system are the line between the solar centre and the solar north pole, as is standard, and the line along the solar equator which aligns to the Earth, *i.e.* if the Earth were to orbit perfectly aligned to the solar equator, the line would always travel through the Earth.
The practical effect of this definition is that the heliographic-stonyhurst system has its north pole at the solar north pole and the 0th meridian of longitude is aligned to the Earth.

### Heliographic (Carrington)

This system differs from the heliographic-stonyhurst system in its definition of the 0th meridian of longitude, in the heliographic-carrington system the meridian rotates around the Sun once every 27 days and aligns with the heliographic-stonyhurst system at the start of each [Carrington rotation]().

In [ ]:
blank = np.zeros((2048, 2048)) * np.NaN

hpc_wcs = WCS(naxis=2)
hpc_wcs.wcs.ctype = ['HPLN-ZEA', 'HPLT-ZEA']
hpc_wcs.wcs.crval = [0., 0.]
hpc_wcs.wcs.crpix = np.array(blank.shape)/2
hpc_wcs.wcs.cdelt = [250.,250.]
hpc_wcs.wcs.cunit = ['arcsec']*2

fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, projection=hpc_wcs)
ax.imshow(blank, origin='lower')

x = ax.coords[0]
y = ax.coords[1]

x.set_axislabel("Longitude [deg]")
y.set_axislabel("Latitude [deg]")

ax.coords.grid(color='black', ls='-')

def update(i, ax, nframes):
    scale = float(nframes-i)*(250./nframes)
    scale+=10 #  Prevent it hitting 0
    
    wcs = copy.deepcopy(ax.wcs)
    wcs.wcs.cdelt = [scale]*2
    ax.reset_wcs(wcs)
    ax.coords.grid(color='black', ls='-')

nframes = 10
ani = anim.FuncAnimation(fig, update, frames=range(nframes), fargs=(ax, nframes))
ani.save("test.mpv")